# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("readWrite2").getOrCreate()
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [2]:
path =  "Datasets/"

pga_data = spark.read.csv(path + "pga_tour_historical.csv", inferSchema=True,header=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [3]:
pga_data.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



In [4]:
pga_data.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [5]:
pga_data.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): https://spark.apache.org/docs/latest/sql-reference.html#data-types

In [6]:
pga_data_edit = pga_data.withColumn("Value", pga_data["Value"].cast('integer'))
pga_data_edit.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [7]:
pga_data_edit.describe('Value').show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           2347998|
|   mean| 12249.25599595911|
| stddev|179434.15932161282|
|    min|              -447|
|    max|           9982040|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [9]:
pga_data_edit.describe('Value','Season').show()

+-------+------------------+------------------+
|summary|             Value|            Season|
+-------+------------------+------------------+
|  count|           2347998|           2740403|
|   mean| 12249.25599595911| 2013.973479083186|
| stddev|179434.15932161282|2.6070501155146517|
|    min|              -447|              2010|
|    max|           9982040|              2018|
+-------+------------------+------------------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [11]:
df = pga_data_edit.select('Value','Season')
df.write.mode("overwrite").parquet("pga_parquet")

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [12]:
df = pga_data_edit.select('Value','Season')
df.write.mode("overwrite").partitionBy('Season').parquet("pga_partitioned_parquet")

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [15]:
path2= "pga_partitioned_parquet/"
pga_by_season = spark.read.parquet(path2)
pga_by_season.limit(5).toPandas()

,Value,Season
0,71,2010
1,77,2010
2,83,2010
3,54,2010
4,100,2010


## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [16]:
path2= "pga_partitioned_parquet/"
pga_by_season3 = spark.read.parquet(path2 + "Season=2010",
                                   path2 + "Season=2011",
                                   path2 + "Season=2012")
pga_by_season3.limit(5).toPandas()

,Value
0,71
1,77
2,83
3,54
4,100


In [19]:
path2= "pga_partitioned_parquet/"
pga_by_season3 = spark.read.option("basePath",path2).parquet(path2 + "Season=2010",
                                   path2 + "Season=2011",
                                   path2 + "Season=2012")
pga_by_season3.limit(5).toPandas()

,Value,Season
0,71,2010
1,77,2010
2,83,2010
3,54,2010
4,100,2010


]]## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [18]:
values =[('apple',12,'red','Kashmir'),('oarnge',7,'Orange','Nagpur'),('Peach',15,'Peach','Mumbai')]

df = spark.createDataFrame(values,['fruits','quantity','Color','farm'])
df.show()

+------+--------+------+-------+
|fruits|quantity| Color|   farm|
+------+--------+------+-------+
| apple|      12|   red|Kashmir|
|oarnge|       7|Orange| Nagpur|
| Peach|      15| Peach| Mumbai|
+------+--------+------+-------+



## We're done! Great job!